正如 2.4节中所说，求导是⼏乎所有深度学习优化算法的关键步骤。虽然求导的计算很简单，只需要⼀些基本的微积分。但对于复杂的模型，⼿⼯进⾏更新是⼀件很痛苦的事情（⽽且经常容易出错）。

深度学习框架通过⾃动计算导数，即⾃动微分（automatic differentiation）来加快求导。

实际中，根据设计好的模型，系统会构建⼀个计算图（computational graph），来跟踪计算是哪些数据通过哪些操作组合起来产⽣输出。

⾃动微分使系统能够随后反向传播梯度。这⾥，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

### 2.3.1 简单的例子
我们对函数$y = 2X^TX$ 关于列向量 $X$ 求导。⾸先，我们创建变量x并为其分配⼀个初始值。

In [18]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于x的梯度之前，需要⼀个地⽅来存储梯度。

重要的是，我们不会在每次对⼀个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。

注意，⼀个标量函数关于向量x的梯度是向量，并且与x具有相同的形状。

In [19]:
x.requires_grad_(True) # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad # 默认值是None

现在计算y

In [20]:
y = 2 * torch.dot(x, x)
print(y)

tensor(28., grad_fn=<MulBackward0>)


x是⼀个⻓度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。

接下来，通过调⽤反向传播函数来⾃动计算y关于x每个分量的梯度，并打印这些梯度。

In [21]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y = 2X^TX$ 关于X的梯度应为4X

让我们快速验证这个梯度是否计算正确。

In [22]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算x的另一个函数

In [25]:
# 在默认情况下， PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([0., 1., 2., 3.], requires_grad=True)
tensor(6., grad_fn=<SumBackward0>)


tensor([1., 1., 1., 1.])

### 2.5.2 非标量变量的反向传播
当y不是标量时，向量y关于向量x的导数的最⾃然解释是⼀个矩阵。对于⾼阶和⾼维的y和x，求导的结果可以是⼀个⾼阶张量。

然⽽，虽然这些更奇特的对象确实出现在⾼级机器学习中（包括深度学习中），但当调⽤向量的反向计算时，我们通常会试图计算⼀批训练样本中每个组成部分的损失函数的导数。

这⾥，我们的⽬的不是计算微分矩阵，⽽是单独计算批量中每个样本的偏导数之和。

In [26]:
# 对⾮标量调⽤backward需要传⼊⼀个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递⼀个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])